# Introduction to Bedrock - Code Generation - Natural Language to SQL 

--- 

In this demo notebook, we demonstrate how to use the Bedrock Python SDK for a code generation example. We show how to use Bedrock Foundational Models to create SQL statements to query data. 

---

Note: This notebook was tested in Amazon SageMaker Studio with Python 3 (Data Science 2.0) kernel.

1. [Set Up](#1.-Set-Up-and-API-walkthrough)
2. [Generating SQL Queries](#2.-Generating-SQL)

## 1. Set Up and API walkthrough

---
Before executing the notebook for the first time, execute this cell to add bedrock extensions to the Python boto3 SDK

---

#### Now let's set up our connection to the Amazon Bedrock SDK using Boto3

In [1]:
import boto3
import json
bedrock = boto3.client(service_name='bedrock',region_name='us-east-1',endpoint_url='https://bedrock.us-east-1.amazonaws.com')

#### We can validate our connection by testing out the _list_foundation_models()_ method, which will tell us all the models available for us to use 

In [2]:
bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': '99a1dc34-9b4e-45fd-9a5a-6ee13df8629b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 12 Jul 2023 12:32:11 GMT',
   'content-type': 'application/json',
   'content-length': '861',
   'connection': 'keep-alive',
   'x-amzn-requestid': '99a1dc34-9b4e-45fd-9a5a-6ee13df8629b'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-e1t-medium',
   'modelId': 'amazon.titan-e1t-medium'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/stability.stable-diffusion-xl',
   'modelId': 'stability.stable-diffusion-xl'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-grande-instruct',
   'modelId': 'ai21.j2-grande-instruct'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-jumbo-instruct',
   'modelId': 'ai21.j2-jumbo-in

#### In this Notebook we will be using the invoke_model() method of Amazon Bedrock. This will be the primary method we use for most of our Text Generation and Processing tasks. 

##### The mandatory parameters required to use this method are, where _modelId_ represents the Amazon Bedrock model ARN, and _body_ which is the prompt for our task. The _body_ prompt will change depending on the foundational model provider selected. We walk through this in detail below

```
{
   modelId= model_id,
   contentType= "application/json",
   accept= "application/json",
   body=body
}

```

## 2. Generating SQL

#### Let's now try out the Amazon Bedrock models to write some SQL Queries on some sample data

In [3]:
prompt_data ="""I have a table called patient with fields ID, AGE, WEIGHT, HEIGHT. Write me a SQL Query which will return the entry with the highest age"""#If you'd like to try your own prompt, edit this parameter!

### Let's start by using the Amazon Titan Large Model

In [4]:
body = json.dumps({"inputText": prompt_data})
modelId = 'amazon.titan-tg1-large' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('results')[0].get('outputText'))


SELECT * FROM patient ORDER BY age DESC LIMIT 1


### Let's try the prompt with the Anthropic Claude Instant Model on Bedrock

In [6]:
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = 'anthropic.claude-instant-v1' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('completion'))

 from the patient table.

```sql
SELECT * FROM patient 
WHERE AGE = (SELECT MAX(AGE) FROM patient)
```

This query will:

1. Select all columns (*) from the patient table 
2. Filter the results where the AGE is equal to the maximum AGE in the patient table. 
3. The subquery (SELECT MAX(AGE) FROM patient) finds the maximum AGE from the patient table.
4. So the outer query will return the row with the highest AGE.

Here is an example of the patient table data:

| ID  | AGE | WEIGHT | HEIGHT |
|:-- |:-- | :--| :--| 
| 1 | 20 | 70 | 180 |
| 2 | 35 | 80 | 170|
| 3 | 65 | 75 | 165 |

The query will return:

| ID  | AGE | WEIGHT | HEIGHT |
|:-- |:-- | :--| :--|
| 3 | 65 | 75 | 165 |

Since 65 is the highest age in the table.

Hope this helps! Let me know if you have any other questions.


### Let's try the prompt with the Jurrasic Grande Model on Bedrock

In [7]:
body = json.dumps({"prompt": prompt_data})
modelId = 'ai21.j2-grande-instruct' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('completions')[0].get('data').get('text'))


To return the entry with the highest age from the `Patient` table using SQL, you can use the `
